# LLM-Based SQL Query Generator
This notebook demonstrates how to generate SQL queries from natural language using GPT-4.
We will:
1. Load necessary libraries.
2. Set up API keys and model configuration.
3. Define a function to format user input for GPT-4.
4. Generate an SQL query based on a given database schema.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import sqlparse
import sqlite3
import streamlit as st

In [ ]:
# Load API key from .env
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # make sure to have your OPENAI API key saved in the .env file

OPENAI_model = "gpt-4"
openai = OpenAI()   

The `prompt_for` function constructs a structured prompt that guides GPT-4 to convert natural language queries into well-structured SQL queries based on a given database schema.

In [ ]:
# Formatting the user prompt
def prompt_for(db_schema,nlp_query):
    system_prompt = f"""
    You are an assistant which takes an input a query written in natural language
    and converts this into a well structured SQL query.
    The schema of the database is: {db_schema}
    Do not provide or explaination of the query, just the query.
    """
    user_prompt = f"Convert the following request into an SQL query: {nlp_query}"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    return messages

In [ ]:
user_query = "Select all directors born before 1980."
db_schema = "data/spider_data/database/imdb/schema.sql"
db_path = "../data/spider_data/database/imdb/imdb.sqlite"

In [ ]:
def load_db_schema(db_path):
    """Fetches the database schema from SQLite."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
    schema = "\n".join(row[0] for row in cursor.fetchall() if row[0])
    
    conn.close()
    return schema

In [ ]:
load_db_schema(db_path)

In [ ]:
print(prompt_for(load_db_schema(db_path),user_query))

In [ ]:
# Generates an SQL query from natural language input using above defined model
def generate_sql(db_schema,nlp_query):
    messages = prompt_for(db_schema,nlp_query)

    # Call OpenAI
    response = openai.chat.completions.create(
        model=OPENAI_model, messages=messages, temperature=0.1
    )

    return response.choices[0].message.content

In [ ]:
sql_query = generate_sql(load_db_schema(db_path),user_query)
print(sql_query)

In [ ]:
def validate_sql(query):
    try:
        parsed = sqlparse.parse(query)
        if not parsed:
            return False, "Invalid SQL syntax."
        return True, "SQL is valid"
    except Exception as e:
        return False, str(e)
    
def execute_test_query(query, db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute(query)
        conn.commit()
        conn.close()
        return True, "SQL executed successfully"
    except Exception as e:
        return False, str(e)

In [ ]:
valid, msg = validate_sql(sql_query)
print("\nValidation Result:\n", msg)

if valid:
    executed, exec_msg = execute_test_query(sql_query,db_path)
    print("\nExecution Test:\n", exec_msg)

In [ ]:
st.title("LLM-Based SQL Query Generator")

st.sidebar.header("Project Settings")
db_path = st.sidebar.text_input("Database Path","imdb.sqlite")

natural_language_query = st.text_area("Enter your natural language query: ", "")